In [1]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append("/home/jbc0071/Documents/CANIDS/code")
# sys.path.append(parent_dir + "/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)

config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)

Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure
No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in rever

In [2]:
unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
model_path = 'models/canolo_model_112.pt'

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}

In [3]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect_wrapper, get_scores_wrapper
import time

num_attack_files = len(attack_loaders)
detectors = []

threshold = 2.80E-06

start_time = time.time()
for _ in range(num_attack_files):
    detector = CANnoloAttackDetector(model_path, threshold, model_config, force_cpu=True)
    detectors.append(detector)

with ProcessPoolExecutor() as executor:
    list_of_filename_scores_and_labels = list(executor.map(get_scores_wrapper, zip(attack_loaders, detectors)))

for loader in attack_loaders:
    loader.reset()
end_time = time.time()

print(f"Finished in {end_time - start_time} seconds")



Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Testing on correlated_signal_attack_1.log
Testing on correlated_signal_attack_2.log
Testing on correlated_signal_attack_3.log
Testing on fuzzing_attack_1.log
Testing on fuzzing_attack_2.logTesting on fuzzing_attack_3.log

Testing on max_engine_coolant_temp_attack.log
Testing on max_speedometer_attack_1.log
Testing on max_speedometer_attack_2.log
Testing on reverse_light_off_attack_1.logTesting on max_speedometer_attack_3.log

Testing on reverse_light_off_attack_2.log
Testing on reverse_light_off_attack_3.log
Testing on reverse_light_on_attack_1.log
Testing on reverse_light_on_attack_3.log
Testing on reverse_light_on_attack_2.log
Finished in 204.87332892417

NameError: name 'list_of_filename_and_results' is not defined

In [15]:
# list_of_filename_scores_and_labels[0]
new_file_format = [[filename, score_list] for filename, score_list in list_of_filename_scores_and_labels]


In [18]:
new_file_format_2 = []
for filename, score_list in new_file_format:
    score_list_new = []
    for score, label in score_list:
        score_list_new.append([score.item(), int(label)])
    new_file_format_2.append([filename, score_list_new])

with open("fileandscore.txt", 'w') as f:
    f.write(str(new_file_format_2))


In [ ]:
new_file_format_2